Na tarefa anterior se fixo unha pequena base de datos con caras. Neste caso se trata de facer unha aplicación de consola que cargue esa base de datos e tente identificar as caras presentes.

Á saída debera escribir os nomes das persoas que foi capaz de identificar e o número de persoas que non foi capaz de poñerlle nome.